 #                 [Lab 03] Analise de sentimentos do Twitter

Projeto que tenta criar um modelo de analise de sentimento com base em tweets com emojis, e dai tentar utilizar esse classificador em outros contextos. O caso escolhido seria saber se ele se enquadraria para analisar o sentimento dos eleitores em relação ao Bolsonaro, após sua entrevista ao roda viva.



In [80]:
import string
import re,string
import pandas as pd
import numpy as np
import nltk
from collections import Counter
import operator
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

### Lendo dados

Foram coletados quase 60 mil twittes com emojis e classificados como positivos ou negativos, com base nos emojis utilizados. 

Os dados estao no arquivo db.csv e eh utilizado o codigo abaixo para ler esse arquivo.

In [81]:
df = pd.read_csv('db.csv', sep='\t') 


### Coletando uma amostra

Para que possamos trabalhar de forma eficiente (tempo de execucao, memoria), coletamos uma amostra desse data frame

In [82]:
df_0 = df[df['sentiment'] == 0].sample(n = int(0.1*df.shape[0])) #pega 10% dos tweets negativos
df_1 = df[df['sentiment'] == 1].sample(n = int(0.1*df.shape[0])) #pega 10% dos tweets positivos

df = df_0.append(df_1)


### Sera necessario fazer uma limpeza nos tweets

Abaixo estao alguns metodos para limpar os tweets antes de processa-los

In [83]:
def remove_punctuation(text):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)

def remove_mention(text):
    new_text = ""
    list_of_words = text.split(" ")
    for word in list_of_words:
        if word[0] != "@":
            new_text = new_text + word + " "
    return new_text.rstrip()

def remove_links(text):
    link_regex    = re.compile('((https?):((//)|(\\\\))+([\w\d:#@%/;$()~_?\+-=\\\.&](#!)?)*)', re.DOTALL)
    links         = re.findall(link_regex, text)
    for link in links:
        text = text.replace(link[0], ', ')    
    return text

def remove_pics(text):
    new_text = ""
    list_of_words = text.split(" ")
    for word in list_of_words:
        if len(word) < 32:  ##ver a representacao da imagem melhor para evitar de remover outras palavras
            new_text = new_text + word + " "
    return new_text.rstrip()

def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False
    
def remove_numbers(text):
    new_text = ""
    list_of_words = text.split(" ")
    for word in list_of_words:
        if not(is_number(word)) :
            new_text = new_text + word + " "
    return new_text.rstrip()

def clean_tweet(text):
    return remove_numbers(remove_punctuation(remove_links(remove_pics(remove_mention(text)))))


### Indice invertido

Cria o Indice invertido, como foi visto no Lab01, para que possa 

In [84]:
#cria um mapa de tokens relacionando o indice do tweet e suas palavras

df['tokens'] = df.apply(lambda row: Counter(nltk.word_tokenize(clean_tweet(row['text'].lower()))), axis=1)

# cria o indice invertido a partir dos tokens gerados

inverted_index = {}

for token_list, _id in zip(df.tokens, df.id):
    for token in token_list.keys():
        if token not in inverted_index.keys():
            inverted_index[token] = [_id]
        else:
            inverted_index[token].append(_id)

### TF-IDF

Funcoes para calcular o TF-IDF de cada termo do Tweet

In [85]:
def idf(term):
    N = df.shape[0] # tamanho do corpus
    return np.log(N/len(inverted_index[term.lower().strip()]))

In [86]:
def tf(term):
    return len(inverted_index[term])

In [87]:
def get_tfidfs(tokens_list):
    resp = {}
    for token in tokens_list:
        resp[token] = tf(token)*idf(token)
    return resp

### Topicos

#### Seleciona as palavras mais relevantes daquele tweet

In [88]:
def get_topics(tokens_list, n= 25):
    tfidfs = get_tfidfs(tokens_list)
    
    sorted_d = sorted(tfidfs.items(), key=operator.itemgetter(1),reverse=True)
    
    return [topic[0] for topic in sorted_d[:n]] 

In [89]:
df['topics'] = df.apply(lambda row: get_topics(row['tokens']), axis=1)

In [90]:
df['topics'] = df.apply(lambda row: get_topics(row['tokens']), axis=1)
df_processed = df[['id','topics', 'sentiment']]

In [91]:
df_processed = df_processed.drop('topics', 1).join(df.topics.str.join('|').str.get_dummies(), lsuffix = "_")

### Dividir os dados

Nesse ponto precisamos de uma porcentagem dos dados para treinar o modelo e outra porcentagem para testar o modelo. Escolhi 80% pra treino e 20% pra teste

In [92]:
df_temp = df_processed[[i for i in list(df_processed.columns) if i not in ['id_', 'sentiment_', "sentiment"]]]

X = df_temp.as_matrix()
y = df_processed[['sentiment']].as_matrix()

In [93]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=34)

## Classificador

Aqui criamos o modelo a partir do algoritmo Naive Bayes Multinomial.

In [94]:
mnb = MultinomialNB()
model_mnb = mnb.fit(X_train, y_train.ravel())

In [95]:
#Uso para teste, para saber qual modelo sai melhor
from sklearn.naive_bayes import BernoulliNB
bnb = BernoulliNB()
model_bnb = bnb.fit(X_train, y_train.ravel())

## Teste do modelo

Com o modelo ja treinado agore precisamos testa-lo.

In [96]:
#definimos 
y_pred = model_mnb.predict(X_test)
y_true = y_test

Usaremos inicialmente a acurácia. Que mostra a porcentagem de acertos/numero total

In [97]:

accuracy_score(y_true, y_pred)

0.7375215146299484

Usaremos também matriz de confusão para saber mais detalhes dos acertos e erros

In [98]:
confusion_matrix(y_true,y_pred)

array([[969, 215],
       [395, 745]])

## Melhoramento o modelo

Antes de chegar nesse resultado do modelo, foi necessário muitos testes e mudanças. 

Minha primeira implementação utilizava o algoritmo Gaussian Naive Bayes para classificação pois foi o primeiro que encontrei que utilizava naive bayes. Também utilizei apenas 5% da base de dados (cerca de 3 mil tweets) e selecionava apenas 3 palavras como tópicos de cada tweet. Foi uma escolha que visava a agilidade de processamento e testes durante a programação. Com essas configurações, a acurácia era pouco mais de 50%. E vendo mais detalhes, utilizando a matriz de confusão, percebi que ele classificava quase todos os tweets como positivo. Por isso acertava todos os positivos e errava quase todos os negativos. 

Imaginei inicialmente que poderia ser o número de tópicos, então resolvi aumentar de 3 para 5. De fato, houve uma melhora na acurácia porém continuava abaixo 55%. Aumentei o número de tópicos para 10, depois 20 e então consegui chegar aos 55%. O problema que por mais que eu aumentasse o número de tópicos, nesse ponto o valor não melhorava. Resolvi então aumentar o número da amostra. Agora coletava 2%, porém nesse ponto comecei a ter problemas de estouro de memória. então vi que não adiantava forçar mais, agora a mudança tinha que ser no modelo ou na forma de tratar os tweets. 

Então fiz diversas limpezas nos tweets, removendo assim mentions, links e imagens. Tudo isso proporcionou uma pequena melhoria, mas nada muito mais do que 66% de acurácia. 

Estava muito insatisfeito com meu modelo. Pensei então em utilizar outro algoritmo, uma rede neural, por exemplo. E nessa pesquisa descobri que havia outras variações do naive bayes. E então utilizando o algoritmo Multinominal Naive Bayes tivemos media de 75% dos acertos e foi possível atingir a marca de 80% de acerto. 

Provavelmente uma melhor limpeza nos tweets, uma melhor classificação (não utilizando apenas emojis para classificar, adicionar uma nova categoria para os neutros) ou utilizar uma amostra maior, seria possível melhorar essa acurácia. Mas a níveis de estudo acredito que foi encontrado um bom modelo.

## Uso do modelo

Agora que temos um bom modelo, gostaria de testa-lo em um conjunto de 30 tweets, classificados à mão por mim,  sobre a entrevista de bolsonaro no roda viva. Vamos ver como nosso modelo sai em outros contextos. 



Precisamos primeiro pegar esses dados e deixa-los na mesma dimensão que o classificador consiga ler.

In [99]:
tweets = ["É difícil escolher a maior barbaridade q Bolsonaro disse no Roda Viva.Mas dizer que os africanos eram os responsáveis pela própria escravidão q os martirizou e q os portugueses ñ pisaram na África ,mostra a covardia explícita deste racista ao tentar transformar a vítima em algoz.",
"se depois dessa entrevista do bolsonaro no roda viva vc ainda vai votar nele na moral só digo uma coisa: da unfollow e me corta pq tem que ser MUITO BURRO, não precisa ter nem 2 neuronios",
"Dps do Bolsonaro no roda viva ficou claro que quem apoia ele tem algum problema mental",
"É muita covardia comparar essa entrevista do Ciro Gomes com a palhaçada que se deu no Roda Viva com Bolsonaro. Não entendo como alguém pode cogitar aquela mula, comparado ao Ciro ou a qualquer outro candidato.",
"Vi a entrevista do Bolsonaro pro roda viva e fui ler os comentários, fiquei horrorizada com o tanto de gente que o apoia, não sei mais o que vai virar",
"Eu sinto tanto nojo dos vídeos do Bolsonaro no Roda Viva que não consigo nem compartilhar pra mostrar o quanto ele é uma merda Não é possível esse cara ter tanto fã burro meu pai do céu",
"Você viu o Bolsonaro pagando mico no RodaViva? Elege ele pra ver a vergonha internacional que nós iremos passar!",
"Bolsonaro no Roda Viva se mostrou totalmente incapaz de governar algo, ele não tá apto nem pra gerir uma bodega quem dera um país. Fora o conhecimento zero em história, o saudosismo a ditadura passando pano pra tortura e os assassinatos, e outras baboseiras ditas, ele é um asno.",
"o indivíduo que assistiu o roda viva com bolsonaro e ainda assim ESCOLHE votar nele tem mais é que receber o título de otário mesmo",
"eu to chocada com as respostas que o bolsonaro deu no roda viva mano MANO M A N O como, COMO, C O M O alguém consegue ver um bom futuro pro nosso país com esse cara na presidência? PUTA MERDA",
"Depois das aberrações de ontem de Bolsonaro, é ainda mais claro que a educação pública precisa incluir a defesa dos direitos humanos e da democracia. Sem acertar contas com o passado, não há presente nem futuro. Como se diz em psicanálise, é um passado que não passa.",
"confesso q não consegui assistir a entrevista inteira do Bolsonaro no Roda Viva.me faz muito mal ouvir aquele ser odioso falar, faz muito mal pro Brasil darem voz pra aquilo, é péssimo pro povo brasileiro, já tão sofrido, sentir o ódio se propagando enquanto o amor segue preso.",
"O #RodaViva de ontem mostrou novamente que não precisamos perder nosso tempo difamando o bolsonaro. Ele já faz isso muito bem sozinho.¯\_(ツ)_/¯",
"Eu creio que o Bolsonaro no #RodaViva diminuiu ainda mais as chances dele vencer alguma coisa Você pode ser ""anti-esquerda"", mas você não é idiota a ponto de votar em alguém claramente despreparado Fale o que quiser, mas não diga que ele é preparado para ser presidente.",
"Assistindo o #RodaViva com o Bolsonaro e não consigo sentir nada além de medo. Medo de ver pessoas apoiando alguém desequilibrado apenas por estarem desesperados por mudanças. Medo ao imaginar alguém como ele governando um país.",
"Os jornalistas do Roda Viva deveriam descer do pedestal pq o Bolsonaro se saiu muito melhor que eles. #OsPingosNosIs",
"Pensando na maneira que foi conduzida a última entrevista no Roda Viva será mesmo que nosso candidato Bolsonaro deverá participar nas demais?! O que vimos foi um bando de abutres tentando desqualifica-lo e ele tem mto mais a oferecer do que ficar dando aula de história!!",
"O Bolsonaro foi perfeito em todas as questões, os esquerdoburros do @rodaviva não conseguiram sequer pegar a placa da jamanta que os atropelou! 😂😂😂",
"ESPETACULAR!!! RECORDE QUE NEM ESSE DUVIDO Q OUTRO POLÍTICO BATERÁ NOS PRÓXIMOS 50 ANOS! Em menos de 48 horas, o vídeo do @jairbolsonaro no Roda Viva acabou de ultrapassar a absurda marca de 4 MILHÕES DE VISUALIZAÇÕES (+1 milhão em 20 horas) e AINDA CONTINUA EM ALTA no YouTube!",
"Roda viva mostrou a face mais estúpida de um jornalismo descomprometido com a verdade, totalmente amador e ideológico. Simplesmente ridículo. Um tribunal com juízes de araque. Bolsonaro só ganhou. De novo.",
"Paulo Figueiredo resumiu “vocês viram o Bolsonaro no Roda Viva? Eu vi um brasileiro comum falando verdades a uma classe jornalística estúpida, ideológica, vagabunda, despreparada e soberba. Poucas vezes vi algo tão ilustrativo do momento em que vivemos “ #BolsonaroNoRodaViva",
"Jair Bolsonaro sai fortalecido do programa Roda Viva. O método de inserir apenas militantes para derrubá-lo foi um tiro no pé. Militantes são monótonos, possuem retórica limitada e não têm compromisso com a realidade dos fatos.",
"Jair Bolsonaro 7 x 1 Roda Viva  A grande mídia precisa refletir, não da mais para vender MILITANTE como JORNALISTA, este truque não engana mais ninguém ...",
"Falam que a gente é burro por apoiar Bolsonaro, mas apoia o socialismo e ainda diz que Lula é inocente. Não é que a gente seja burro, são vocês que não aceitam a REALIDADE.  É @jairbolsonaro 2018 sim, se não gostou vai pra cuba.",
"Resumo do #RodaViva com BOLSONARO -Ditadura -Ditadura -Tortura -Quilombolas -Jesus era refugiado -Wikipedia -Ditadura - Exército Saúde? NÃO Saneamento básico? NÃO. Educação? NÃO.Segurança? NÃO.Não vimos uma entrevista. Vimos um folhetim esquerdista baseado na WIKIPEDIA!",
"O Roda Viva empurrou mais a faixa para o Jair. @jairbolsonaro é uma espécie de Rocky Balboa, quanto mais apanha, levanta mais forte e motivado.",
"É visível o ódio desses esquerdopatas pelo Bolsonaro. Se Deus quiser, Bolsonaro presidente,",
"Porra hein, achei muita sacanagem esses jornalixos enchendo o saco do Bolsonaro, sinceramente nem acho que ele precisava disso, deve ser muito estressante pra ele 😤😤😡😡",
"Fizeram de propósito só passado e mimimi para boicotar a entrevista",
"Meu caro, você mitou grandemente! 😍"]

In [100]:
tweets_processed = [tweet.split() for tweet in tweets ]

In [101]:
tweets_test = []

num_columns_to_train = X_train.shape[1]

for tweet_topics in tweets_processed:
    row = np.zeros((df_temp.shape[1],), dtype=int)
    
    #fill the row
    for topic in tweet_topics:
        if topic in (df_processed.columns.values):
            idx = df_processed.columns.get_loc(topic)
            row[idx] = 1
    
    tweets_test.append(row)

Agora passaremos esses novos tweets para o classificador e coletar o resultado.

In [102]:
result = model_mnb.predict(tweets_test)
y_real = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]

In [103]:
result

array([1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1])

Agora vamos ver os resultados

In [104]:
accuracy_score(y_real, result)

0.5333333333333333

In [105]:
confusion_matrix(y_real,result)

array([[ 3, 12],
       [ 2, 13]])

## Conclusao

Podemos ver, pela acurácia, que o modelo funciona muito mal fora do seu contexto. Talvez reflita um pouco o tom e palavras ironicos e agressivas que os usuários, de ambos os lados, utilizam ao falar sobre politica e Bolsonaro. 
Também podemos observar, pela matriz de confusão, que o modelo erra mais ao julgar que os tweets são negativos quando na verdade são positivos. 

Algo curioso, que se nota ao rodar esse notebook diversas vezes, é o fato de que, quando há uma melhora na acurácia dos testes usando o mesmo contexto do treino, diminui a acurácia dos testes com os tweets sobre Bolsonaro. Isso pode nos mostrar sobre o quanto que é positivo e negativo em um contexto pode ser o oposto em outro.

Podemos então concluir, que para termos melhores resultados no nosso classificador devemos treina-lo de acordo com  precisamos treinar o modelo em um contexto mais específico para termos melhores resultados.